# VER 2.0 Option B - CV-Based Training

**Key Difference:** Training data generated from `MechanisticSimulator` (realistic CV curves)
vs Option A's pure Gaussian summation.

---

In [ ]:
# Upload training data
from google.colab import files
print('Upload training_data_cv.npz:')
uploaded = files.upload()
DATA_PATH = 'training_data_cv.npz'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

print(f'TensorFlow: {tf.__version__}')
print(f'GPU: {tf.config.list_physical_devices("GPU")}')

In [ ]:
# Load CV-based dataset
data = np.load(DATA_PATH)
signals = data['signals']
heatmaps = data['heatmaps']
voltage_grid = data['voltage_grid']

print(f'Signals shape: {signals.shape}')
print(f'Heatmaps shape: {heatmaps.shape}')
print(f'Signal range: [{signals.min():.1f}, {signals.max():.1f}] µA')

In [ ]:
# Normalize
signal_mean = signals.mean()
signal_std = signals.std()
signals_norm = (signals - signal_mean) / signal_std

X = signals_norm.reshape(-1, 500, 1)
Y = heatmaps.reshape(-1, 500, 1)

split_idx = int(0.9 * len(X))
X_train, X_val = X[:split_idx], X[split_idx:]
Y_train, Y_val = Y[:split_idx], Y[split_idx:]
print(f'Train: {len(X_train)}, Val: {len(X_val)}')

In [ ]:
# Same U-Net architecture
def build_unet_1d(input_shape=(500, 1)):
    inputs = keras.Input(shape=input_shape)
    c1 = layers.Conv1D(32, 7, padding='same', activation='relu')(inputs)
    c1 = layers.Conv1D(32, 7, padding='same', activation='relu')(c1)
    p1 = layers.MaxPooling1D(2)(c1)
    c2 = layers.Conv1D(64, 5, padding='same', activation='relu')(p1)
    c2 = layers.Conv1D(64, 5, padding='same', activation='relu')(c2)
    p2 = layers.MaxPooling1D(2)(c2)
    c3 = layers.Conv1D(128, 3, padding='same', activation='relu')(p2)
    c3 = layers.Conv1D(128, 3, padding='same', activation='relu')(c3)
    p3 = layers.MaxPooling1D(5)(c3)
    c4 = layers.Conv1D(256, 3, padding='same', activation='relu')(p3)
    c4 = layers.Conv1D(256, 3, padding='same', activation='relu')(c4)
    u3 = layers.UpSampling1D(5)(c4)
    u3 = layers.Concatenate()([u3, c3])
    d3 = layers.Conv1D(128, 3, padding='same', activation='relu')(u3)
    d3 = layers.Conv1D(128, 3, padding='same', activation='relu')(d3)
    u2 = layers.UpSampling1D(2)(d3)
    u2 = layers.Concatenate()([u2, c2])
    d2 = layers.Conv1D(64, 5, padding='same', activation='relu')(u2)
    d2 = layers.Conv1D(64, 5, padding='same', activation='relu')(d2)
    u1 = layers.UpSampling1D(2)(d2)
    u1 = layers.Concatenate()([u1, c1])
    d1 = layers.Conv1D(32, 7, padding='same', activation='relu')(u1)
    d1 = layers.Conv1D(32, 7, padding='same', activation='relu')(d1)
    outputs = layers.Conv1D(1, 1, activation='sigmoid')(d1)
    return keras.Model(inputs, outputs, name='UNet1D_CV')

model = build_unet_1d()
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['mae'])
print(f'Total params: {model.count_params():,}')

In [ ]:
# Train
callbacks = [
    keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
]
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),
                    epochs=50, batch_size=64, callbacks=callbacks)

In [ ]:
# Plot training
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Val')
axes[0].set_title('Loss'); axes[0].legend()
axes[1].plot(history.history['mae'], label='Train')
axes[1].plot(history.history['val_mae'], label='Val')
axes[1].set_title('MAE'); axes[1].legend()
plt.show()

In [ ]:
# Evaluate
fig, axes = plt.subplots(5, 3, figsize=(15, 16))
for i in range(5):
    idx = np.random.randint(len(X_val))
    signal = X_val[idx:idx+1]
    true_hm = Y_val[idx].squeeze()
    pred_hm = model.predict(signal, verbose=0).squeeze()
    sig_display = signal.squeeze() * signal_std + signal_mean
    axes[i,0].plot(voltage_grid, sig_display, 'b-')
    axes[i,0].set_title('CV Signal'); axes[i,0].grid(True, alpha=0.3)
    axes[i,1].fill_between(voltage_grid, true_hm, alpha=0.5, color='green')
    axes[i,1].set_title('True Heatmap'); axes[i,1].set_ylim(0,1.1)
    axes[i,2].fill_between(voltage_grid, pred_hm, alpha=0.5, color='orange')
    axes[i,2].set_title('Predicted'); axes[i,2].set_ylim(0,1.1)
plt.tight_layout(); plt.show()

In [ ]:
# Save
np.savez('normalization_params_cv.npz', signal_mean=signal_mean, signal_std=signal_std, voltage_grid=voltage_grid)
model.save('peak_detector_cv.keras')
print('Saved!')
files.download('peak_detector_cv.keras')
files.download('normalization_params_cv.npz')